In [60]:
# Login
import wrds
db = wrds.Connection(wrds_username="hanyuzhang")
# password is Timeseries2020!

Enter your WRDS username [hzhang]:hanyuzhang
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [61]:
# Preprocess Data
import pandas as pd
sym_df = pd.read_csv("Ticker.csv", header=0)
sym_list = sym_df["Ticker"].tolist()
sym_permno_list = list()
for symbol in sym_list:
    if len(symbol) == 0:
        continue
    result = db.raw_sql(f"""select permno, htsymbol 
                           from crsp.dsfhdr 
                           where htsymbol = '{symbol}'""")
    try:
        sym_permno_list.append((result.iloc[0]['permno'], result.iloc[0]['htsymbol']))
    except:
        print(f"wrds doesn't have data for {symbol} right now, skip...")

wrds doesn't have data for RTX right now, skip...


In [62]:
from collections import defaultdict
timeseries_bank = defaultdict(list)
for permno, symbol in sym_permno_list:
    result = db.raw_sql(f"""select date, prc, cfacpr
                            from crsp.dsf
                            where permno = {permno} and date > '2010-01-01'
                         """)
    ts = {date:prc for date, prc in zip(result["date"].tolist(), (result["prc"]/result["cfacpr"]).tolist())}
    timeseries_bank[symbol] = ts  

In [63]:
# Build pairs
sym_pairs = list()
sym_pairs_ts = dict()
valid_sym_list = list(timeseries_bank.keys())
for i in range(len(valid_sym_list)-1):
    for j in range(i+1, len(valid_sym_list)):
        sym_pairs.append((valid_sym_list[i], valid_sym_list[j]))
for pair in sym_pairs:
    sym_pairs_ts[pair] = {"first":timeseries_bank[pair[0]], "second":timeseries_bank[pair[1]]}
    
# sym_paris_ts is the pair timeseries:
# {(sym1, sym2):{'first':[(datetime.date(2010,1,1),price1), (datetime.date(2010,1,2):price2), ...}, "second":[...]}}

In [68]:
# Back Test Framework
# utility function
def cal_spread(first, second)->float:
    pass

def test(sym1_ts, sym2_ts, start_date, sigma, mean, half_life):
    open_price = 0
    close_price = 0
    opened = False
    open_date = None
    from dateutil.relativedelta import relativedelta
    end_date = start_date + relativedelta(months=3)
    curr_date = start_date
    while curr_date < end_date:
        if curr_date not in sym1_ts or curr_date not in sym2_ts:
            curr_date += relativedelta(days=1)
            continue
        spread = cal_spread(sym1_ts[curr_date], sym2_ts[curr_date])
        if spread > 2*sigma + mean and end_date - curr_date > datetime.timedelta(days = 2 * half_life) and not opened:
            opened = True
            open_date = curr_date
            open_price = sym1_ts[curr_date]-sym2_ts[curr_date]
        # trade close
        # add factor correlation?
        elif opened and (spread <= mean or curr_date - open_date > datetime.timedelta(days = 3 * half_life)):
            close_price = sym1_ts[curr_date]-sym2_ts[curr_date]
            opened = False
            break
        curr_date += relativedelta(days=1)
    if opened:
        curr_date -= relativedelta(days=1)
        close_price = sym1_ts[curr_date]-sym2_ts[curr_date]
        opened = False
    return close_price - open_price            